In [1]:
import math
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import lxml
import re
from random import seed
from collections import Counter
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import array
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

C:\Users\MoJo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LinkOutdoor4x100m = "http://www.ustfccca.org/records-lists/outdoor-collegiate-all-time-bests?gender=1&sport=2&div=20527&top=25&event=18"

In [3]:
page = requests.get(LinkOutdoor4x100m)
soup = BeautifulSoup(page.text,'html')
soup

table = soup.find("table")

In [4]:
Header = soup.findAll('th')
Headers = []
for h in Header:
    Headers.append(h.text)

In [5]:
Headers

['Team', 'Institution', 'Mark', 'Date', '']

In [6]:
row_soup = soup.findAll('tr')

In [7]:
row_soup

[<tr><th style="width: 35%;">Team</th><th colspan="2" style="width: 30%;">Institution</th><th style="width: 25%;">Mark</th><th style="width: 10%;">Date</th><th></th></tr>,
 <tr><td colspan="100" style="font-weight: bolder;">UNOFFICIAL Top 25 Performers and Performances</td></tr>,
 <tr><td colspan="5"><div class="container-table" style="width: 100%;"><table class="embed" width="100%"><thead><tr></tr></thead><tr><td style="width: 35%;"><span>(1) LSU</span></td><td style="width: 5%;"><span class="group-logo-inline"><img alt="LSU" class="group-logo-inline" src="/images/logos/rankings-article/lsu.png" title="LSU"/></span></td><td style="width: 25%;"> </td><td style="width: 25%;"><a href="https://www.tfrrs.org/results/80833/4927327" target="_blank">37.90</a><a href="https://www.youtube.com/watch?v=odyzo7pVpas" target="_blank">📺</a> </td><td style="width: 10%;text-align:right;" title="SEC Outdoor Championships (Baton Rouge, La.)"><span class="mark-date" style="width: 10%;text-align:right;back

In [8]:
rows = []
for row in row_soup:
    row_data = []
    for cell in row.findAll('td'):
        row_data.append(cell.text)
    rows.append(row_data)

In [9]:
rows

[[],
 ['UNOFFICIAL Top 25 Performers and Performances'],
 ["(1) LSU\xa037.90📺 5/13/2023\xa0\xa0\xa0\xa0\xa0Brandon HICKLIN, Dorian CAMEL, Da'Marcus FLEMING, Godson OGHENEBRUME",
  '(1) LSU',
  '',
  '\xa0',
  '37.90📺 ',
  '5/13/2023',
  "\xa0\xa0\xa0\xa0\xa0Brandon HICKLIN, Dorian CAMEL, Da'Marcus FLEMING, Godson OGHENEBRUME"],
 [],
 ['(1) LSU', '', '\xa0', '37.90📺 ', '5/13/2023'],
 ["\xa0\xa0\xa0\xa0\xa0Brandon HICKLIN, Dorian CAMEL, Da'Marcus FLEMING, Godson OGHENEBRUME"],
 ["(2) Texas Tech\xa037.93 6/9/2023\xa0\xa0\xa0\xa0\xa0Adam CLAYTON, Courtney LINDSEY, Don'dre SWINT, Terrence JONES",
  '(2) Texas Tech',
  '',
  '\xa0',
  '37.93 ',
  '6/9/2023',
  "\xa0\xa0\xa0\xa0\xa0Adam CLAYTON, Courtney LINDSEY, Don'dre SWINT, Terrence JONES"],
 [],
 ['(2) Texas Tech', '', '\xa0', '37.93 ', '6/9/2023'],
 ["\xa0\xa0\xa0\xa0\xa0Adam CLAYTON, Courtney LINDSEY, Don'dre SWINT, Terrence JONES"],
 ['(2) Florida\xa037.93 5/13/2023\xa0\xa0\xa0\xa0\xa0Ryan WILLIE, Jacory PATTERSON, PJ AUSTIN, Robert G

In [10]:
len(rows)

521

In [11]:
rows[499]

['\xa0\xa0\xa0\xa0— AUBURN {6}\xa038.69 6/9/2012\xa0\xa0\xa0\xa0\xa0Jeremy Hardy, Harry Adams, Marcus Rowland, Keenan Brock',
 '\xa0\xa0\xa0\xa0— AUBURN {6}',
 '',
 '\xa0',
 '38.69 ',
 '6/9/2012',
 '\xa0\xa0\xa0\xa0\xa0Jeremy Hardy, Harry Adams, Marcus Rowland, Keenan Brock']

In [12]:
len(rows)

521

In [13]:
RealRows = []

In [14]:
for i in range(0,500):
    if len(rows[i]) > 1:
        RealRows.append(rows[i])
        

In [15]:
RealRows

[["(1) LSU\xa037.90📺 5/13/2023\xa0\xa0\xa0\xa0\xa0Brandon HICKLIN, Dorian CAMEL, Da'Marcus FLEMING, Godson OGHENEBRUME",
  '(1) LSU',
  '',
  '\xa0',
  '37.90📺 ',
  '5/13/2023',
  "\xa0\xa0\xa0\xa0\xa0Brandon HICKLIN, Dorian CAMEL, Da'Marcus FLEMING, Godson OGHENEBRUME"],
 ['(1) LSU', '', '\xa0', '37.90📺 ', '5/13/2023'],
 ["(2) Texas Tech\xa037.93 6/9/2023\xa0\xa0\xa0\xa0\xa0Adam CLAYTON, Courtney LINDSEY, Don'dre SWINT, Terrence JONES",
  '(2) Texas Tech',
  '',
  '\xa0',
  '37.93 ',
  '6/9/2023',
  "\xa0\xa0\xa0\xa0\xa0Adam CLAYTON, Courtney LINDSEY, Don'dre SWINT, Terrence JONES"],
 ['(2) Texas Tech', '', '\xa0', '37.93 ', '6/9/2023'],
 ['(2) Florida\xa037.93 5/13/2023\xa0\xa0\xa0\xa0\xa0Ryan WILLIE, Jacory PATTERSON, PJ AUSTIN, Robert GREGORY',
  '(2) Florida',
  '',
  '\xa0',
  '37.93 ',
  '5/13/2023',
  '\xa0\xa0\xa0\xa0\xa0Ryan WILLIE, Jacory PATTERSON, PJ AUSTIN, Robert GREGORY'],
 ['(2) Florida', '', '\xa0', '37.93 ', '5/13/2023'],
 ['\xa0\xa0\xa0\xa0— FLORIDA {2}\xa037.97 6/7

In [16]:
RealRealRows = RealRows[::2]
RealRealRows

[["(1) LSU\xa037.90📺 5/13/2023\xa0\xa0\xa0\xa0\xa0Brandon HICKLIN, Dorian CAMEL, Da'Marcus FLEMING, Godson OGHENEBRUME",
  '(1) LSU',
  '',
  '\xa0',
  '37.90📺 ',
  '5/13/2023',
  "\xa0\xa0\xa0\xa0\xa0Brandon HICKLIN, Dorian CAMEL, Da'Marcus FLEMING, Godson OGHENEBRUME"],
 ["(2) Texas Tech\xa037.93 6/9/2023\xa0\xa0\xa0\xa0\xa0Adam CLAYTON, Courtney LINDSEY, Don'dre SWINT, Terrence JONES",
  '(2) Texas Tech',
  '',
  '\xa0',
  '37.93 ',
  '6/9/2023',
  "\xa0\xa0\xa0\xa0\xa0Adam CLAYTON, Courtney LINDSEY, Don'dre SWINT, Terrence JONES"],
 ['(2) Florida\xa037.93 5/13/2023\xa0\xa0\xa0\xa0\xa0Ryan WILLIE, Jacory PATTERSON, PJ AUSTIN, Robert GREGORY',
  '(2) Florida',
  '',
  '\xa0',
  '37.93 ',
  '5/13/2023',
  '\xa0\xa0\xa0\xa0\xa0Ryan WILLIE, Jacory PATTERSON, PJ AUSTIN, Robert GREGORY'],
 ['\xa0\xa0\xa0\xa0— FLORIDA {2}\xa037.97 6/7/2019\xa0\xa0\xa0\xa0\xa0Raymond EKEVWO (JR), Hakim SANI BROWN (SO), Grant HOLLOWAY (JR), Ryan CLARK (SR)',
  '\xa0\xa0\xa0\xa0— FLORIDA {2}',
  '',
  '\xa0',

In [17]:
RealRealRows[1][4]

'37.93 '

In [18]:
len(RealRealRows)

127

In [19]:
OfficialTimes = []

In [20]:
for i in range(0,len(RealRealRows)):
    OfficialTimes.append(RealRealRows[i][4])
OfficialTimes

['37.90📺 ',
 '37.93 ',
 '37.93 ',
 '37.97 ',
 '38.05 ',
 '38.06📺 qq = Performance occurred during a non-final round. ',
 '38.08 ',
 '38.17 ',
 '38.20 qq = Performance occurred during a non-final round. ',
 '38.22 qq = Performance occurred during a non-final round. ',
 '38.23 AA = Performance occurred at altitude (>1000 meters above sea level) ',
 '38.24 ',
 '38.24  ',
 '38.26 ',
 '38.26 ',
 '38.27 qq = Performance occurred during a non-final round. ',
 '38.29 ',
 '38.29 ',
 '38.30 ',
 '38.30 ',
 '38.32 qq = Performance occurred during a non-final round. ',
 '38.33 ',
 '38.34 ',
 '38.35 qq = Performance occurred during a non-final round. ',
 '38.35  ',
 '38.36📺 ',
 '38.37 qq = Performance occurred during a non-final round. ',
 '38.38 ',
 '38.38 qq = Performance occurred during a non-final round. ',
 '38.41 ',
 '38.42 ',
 '38.42  ',
 '38.43 qq = Performance occurred during a non-final round. ',
 '38.44 ',
 '38.44  ',
 '38.45 ',
 '38.45 ',
 '38.45  ',
 '38.46  ',
 '38.47 ',
 '38.47 qq = P

In [21]:
RealRealRows[0][1]

'(1) LSU'

In [22]:
Names = []

for i in range(0,len(RealRealRows)):
    Names.append(RealRealRows[i][1])
Names

['(1) LSU',
 '(2) Texas Tech',
 '(2) Florida',
 '\xa0\xa0\xa0\xa0— FLORIDA {2}',
 '\xa0\xa0\xa0\xa0— LSU {2}',
 '\xa0\xa0\xa0\xa0— LSU {3}',
 '(4) Florida State',
 '(5) Houston',
 '\xa0\xa0\xa0\xa0— TEXAS TECH {2}',
 '\xa0\xa0\xa0\xa0— FLORIDA {3}',
 '(6) TCU',
 '\xa0\xa0\xa0\xa0— TEXAS TECH {3}',
 '\xa0\xa0\xa0\xa0— LSU {4}',
 '\xa0\xa0\xa0\xa0— FLORIDA {4}',
 '\xa0\xa0\xa0\xa0— LSU {5}',
 '\xa0\xa0\xa0\xa0— FLORIDA STATE {2}',
 '\xa0\xa0\xa0\xa0— FLORIDA STATE {3}',
 '\xa0\xa0\xa0\xa0— FLORIDA {5}',
 '(7) Texas A&M',
 '(7) Auburn',
 '\xa0\xa0\xa0\xa0— LSU {6}',
 '\xa0\xa0\xa0\xa0— LSU {7}',
 '\xa0\xa0\xa0\xa0— HOUSTON {2}',
 '\xa0\xa0\xa0\xa0— FLORIDA {6}',
 '\xa0\xa0\xa0\xa0— FLORIDA {7}',
 '\xa0\xa0\xa0\xa0— LSU {8}',
 '\xa0\xa0\xa0\xa0— LSU {9}',
 '\xa0\xa0\xa0\xa0— LSU {10}',
 '\xa0\xa0\xa0\xa0— TEXAS A&M {2}',
 '\xa0\xa0\xa0\xa0— LSU {11}',
 '\xa0\xa0\xa0\xa0— LSU {12}',
 '\xa0\xa0\xa0\xa0— LSU {13}',
 '\xa0\xa0\xa0\xa0— FLORIDA STATE {4}',
 '\xa0\xa0\xa0\xa0— HOUSTON {3}',
 '\x

In [23]:
OfficialReGexNames = []

In [24]:
for i in range(0,127):
    if Names[i].startswith("\xa0\xa0"):
        OfficialReGexNames.append(Names[i][4:])
    else:
        OfficialReGexNames.append(Names[i])
        
OfficialReGexNames

['(1) LSU',
 '(2) Texas Tech',
 '(2) Florida',
 '— FLORIDA {2}',
 '— LSU {2}',
 '— LSU {3}',
 '(4) Florida State',
 '(5) Houston',
 '— TEXAS TECH {2}',
 '— FLORIDA {3}',
 '(6) TCU',
 '— TEXAS TECH {3}',
 '— LSU {4}',
 '— FLORIDA {4}',
 '— LSU {5}',
 '— FLORIDA STATE {2}',
 '— FLORIDA STATE {3}',
 '— FLORIDA {5}',
 '(7) Texas A&M',
 '(7) Auburn',
 '— LSU {6}',
 '— LSU {7}',
 '— HOUSTON {2}',
 '— FLORIDA {6}',
 '— FLORIDA {7}',
 '— LSU {8}',
 '— LSU {9}',
 '— LSU {10}',
 '— TEXAS A&M {2}',
 '— LSU {11}',
 '— LSU {12}',
 '— LSU {13}',
 '— FLORIDA STATE {4}',
 '— HOUSTON {3}',
 '— LSU {14}',
 '— TEXAS TECH {4}',
 '— HOUSTON {4}',
 '(9) South Carolina',
 '— TCU {2}',
 '— FLORIDA {8}',
 '— AUBURN {2}',
 '(10) Arkansas',
 '— LSU {15}',
 '— AUBURN {3}',
 '(11) North Carolina A&T',
 '— LSU {16}',
 '(12) Louisville',
 '(12) Southern California',
 '— HOUSTON {5}',
 '— FLORIDA {9}',
 '— ARKANSAS {2}',
 '— TCU {3}',
 '— TEXAS A&M {3}',
 '— FLORIDA STATE {5}',
 '— FLORIDA {10}',
 '— FLORIDA {11}',
 

In [25]:
len(OfficialTimes)

127

In [26]:
len(OfficialReGexNames)

127

In [27]:
CleanedNames = []

for i in range(0,len(RealRealRows)):
    if OfficialReGexNames[i].startswith("—"):
        CleanedNames.append(OfficialReGexNames[i][2:])
    if OfficialReGexNames[i].startswith("("):
        CleanedNames.append(OfficialReGexNames[i][4:])        
CleanedNames


['LSU',
 'Texas Tech',
 'Florida',
 'FLORIDA {2}',
 'LSU {2}',
 'LSU {3}',
 'Florida State',
 'Houston',
 'TEXAS TECH {2}',
 'FLORIDA {3}',
 'TCU',
 'TEXAS TECH {3}',
 'LSU {4}',
 'FLORIDA {4}',
 'LSU {5}',
 'FLORIDA STATE {2}',
 'FLORIDA STATE {3}',
 'FLORIDA {5}',
 'Texas A&M',
 'Auburn',
 'LSU {6}',
 'LSU {7}',
 'HOUSTON {2}',
 'FLORIDA {6}',
 'FLORIDA {7}',
 'LSU {8}',
 'LSU {9}',
 'LSU {10}',
 'TEXAS A&M {2}',
 'LSU {11}',
 'LSU {12}',
 'LSU {13}',
 'FLORIDA STATE {4}',
 'HOUSTON {3}',
 'LSU {14}',
 'TEXAS TECH {4}',
 'HOUSTON {4}',
 'South Carolina',
 'TCU {2}',
 'FLORIDA {8}',
 'AUBURN {2}',
 ' Arkansas',
 'LSU {15}',
 'AUBURN {3}',
 ' North Carolina A&T',
 'LSU {16}',
 ' Louisville',
 ' Southern California',
 'HOUSTON {5}',
 'FLORIDA {9}',
 'ARKANSAS {2}',
 'TCU {3}',
 'TEXAS A&M {3}',
 'FLORIDA STATE {5}',
 'FLORIDA {10}',
 'FLORIDA {11}',
 'LSU {17}',
 'HOUSTON {6}',
 'FLORIDA {12}',
 'AUBURN {4}',
 'TEXAS A&M {4}',
 'HOUSTON {7}',
 'FLORIDA {13}',
 ' Arizona State',
 'FLORID

In [28]:
CleanedNames2 = []

for i in range(0,len(RealRealRows)):
    if CleanedNames[i].startswith(" "):
        CleanedNames2.append(CleanedNames[i][1:])
    else:
        CleanedNames2.append(CleanedNames[i])        
CleanedNames2

['LSU',
 'Texas Tech',
 'Florida',
 'FLORIDA {2}',
 'LSU {2}',
 'LSU {3}',
 'Florida State',
 'Houston',
 'TEXAS TECH {2}',
 'FLORIDA {3}',
 'TCU',
 'TEXAS TECH {3}',
 'LSU {4}',
 'FLORIDA {4}',
 'LSU {5}',
 'FLORIDA STATE {2}',
 'FLORIDA STATE {3}',
 'FLORIDA {5}',
 'Texas A&M',
 'Auburn',
 'LSU {6}',
 'LSU {7}',
 'HOUSTON {2}',
 'FLORIDA {6}',
 'FLORIDA {7}',
 'LSU {8}',
 'LSU {9}',
 'LSU {10}',
 'TEXAS A&M {2}',
 'LSU {11}',
 'LSU {12}',
 'LSU {13}',
 'FLORIDA STATE {4}',
 'HOUSTON {3}',
 'LSU {14}',
 'TEXAS TECH {4}',
 'HOUSTON {4}',
 'South Carolina',
 'TCU {2}',
 'FLORIDA {8}',
 'AUBURN {2}',
 'Arkansas',
 'LSU {15}',
 'AUBURN {3}',
 'North Carolina A&T',
 'LSU {16}',
 'Louisville',
 'Southern California',
 'HOUSTON {5}',
 'FLORIDA {9}',
 'ARKANSAS {2}',
 'TCU {3}',
 'TEXAS A&M {3}',
 'FLORIDA STATE {5}',
 'FLORIDA {10}',
 'FLORIDA {11}',
 'LSU {17}',
 'HOUSTON {6}',
 'FLORIDA {12}',
 'AUBURN {4}',
 'TEXAS A&M {4}',
 'HOUSTON {7}',
 'FLORIDA {13}',
 'Arizona State',
 'FLORIDA {14

In [29]:
Times = []
for i in range(0,len(OfficialTimes)):
    Times.append(OfficialTimes[i][:5])
    
Times

['37.90',
 '37.93',
 '37.93',
 '37.97',
 '38.05',
 '38.06',
 '38.08',
 '38.17',
 '38.20',
 '38.22',
 '38.23',
 '38.24',
 '38.24',
 '38.26',
 '38.26',
 '38.27',
 '38.29',
 '38.29',
 '38.30',
 '38.30',
 '38.32',
 '38.33',
 '38.34',
 '38.35',
 '38.35',
 '38.36',
 '38.37',
 '38.38',
 '38.38',
 '38.41',
 '38.42',
 '38.42',
 '38.43',
 '38.44',
 '38.44',
 '38.45',
 '38.45',
 '38.45',
 '38.46',
 '38.47',
 '38.47',
 '38.47',
 '38.48',
 '38.48',
 '38.48',
 '38.48',
 '38.49',
 '38.49',
 '38.49',
 '38.49',
 '38.49',
 '38.50',
 '38.50',
 '38.51',
 '38.52',
 '38.52',
 '38.53',
 '38.53',
 '38.53',
 '38.53',
 '38.53',
 '38.53',
 '38.53',
 '38.54',
 '38.54',
 '38.54',
 '38.54',
 '38.54',
 '38.54',
 '38.54',
 '38.54',
 '38.55',
 '38.56',
 '38.57',
 '38.57',
 '38.57',
 '38.57',
 '38.57',
 '38.58',
 '38.58',
 '38.58',
 '38.58',
 '38.58',
 '38.59',
 '38.59',
 '38.59',
 '38.59',
 '38.59',
 '38.60',
 '38.60',
 '38.60',
 '38.60',
 '38.60',
 '38.60',
 '38.60',
 '38.61',
 '38.62',
 '38.62',
 '38.63',
 '38.63',


In [30]:
Men4x100mDF = pd.DataFrame({'Names': CleanedNames2, 'OfficialTimes': Times})
Men4x100mDF.to_csv('Men4x100m.csv', index=False)